# Github repo

clone github repository

In [1]:
!git clone https://github.com/thomasmaliappis/context-group-detection.git

Cloning into 'context-group-detection'...
remote: Enumerating objects: 718, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 718 (delta 11), reused 20 (delta 9), pack-reused 688
Receiving objects: 100% (718/718), 1.49 MiB | 7.81 MiB/s, done.
Resolving deltas: 100% (398/398), done.


move in repository folder

In [2]:
%cd context-group-detection

/content/context-group-detection


change branch

In [3]:
!git checkout dante

Branch 'dante' set up to track remote branch 'dante' from 'origin'.
Switched to a new branch 'dante'


# Drive

mount drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Requirements
install missing packages

In [5]:
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 6.7 MB/s eta 0:00:00


In [6]:
!pip install pykalman

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.9/228.9 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pykalman: filename=pykalman-0.9.5-py3-none-any.whl size=48442 sha256=e72ace93d2b7e008c3ec76a5be4f356ebb8b024d8b5c20336ac27edae15cda39
  Stored in directory: /root/.cache/pip/wheels/76/33/ef/5f332226e13a5089c6dd4b01cc2bcb59491d18f955fa2d3807
Successfully built pykalman


# Imports

In [8]:
import argparse

import numpy as np
import tensorflow as tf
from keras.callbacks import EarlyStopping, TensorBoard
from keras.layers import Dense, Conv1D, LSTM, concatenate, Input, Flatten
from keras.models import Model
from keras.optimizers import Adam

from models.utils import ValLoss, load_dataset, save_model_data

# Functions

create model

In [9]:
def build_model(context_size, consecutive_frames, features, units, reg_amount, drop_amount, learning_rate):
    """
    Builds model based on given parameters.
    :param context_size: size of context
    :param consecutive_frames: number of frames per scene
    :param features: features
    :param units: units to be used in filters
    :param reg_amount: regularization factor
    :param drop_amount: dropout rate
    :param learning_rate: learning rate
    :return: model
    """
    inputs = []

    # pair branch
    # create input layers
    pair_inputs = []
    for i in range(2):
        pair_input = Input(shape=(consecutive_frames, features), name='pair_{}'.format(i))
        pair_inputs.append(pair_input)
        inputs.append(pair_input)

    pair_layers = []
    for pair_input in pair_inputs:
        lstm = LSTM(64, return_sequences=True)(pair_input)
        pair_layers.append(lstm)

    # reg = l2(reg_amount)

    pair_concatenated = concatenate(pair_layers)
    # pair_reshaped = Reshape((pair_concatenated.shape[1], 1))(pair_concatenated)
    pair_conv = Conv1D(filters=32, kernel_size=3, activation='relu', name='pair_conv')(pair_concatenated)
    # drop = Dropout(drop_amount)(pair_conv)
    # batch_norm = BatchNormalization()(drop)
    # max_pool = MaxPooling1D()(batch_norm)
    # drop = Dropout(drop_amount)(max_pool)
    # batch_norm = BatchNormalization()(drop)
    pair_layer = pair_conv

    # context branch
    context_inputs = []
    for i in range(context_size):
        context_input = Input(shape=(consecutive_frames, features), name='context_{}'.format(i))
        context_inputs.append(context_input)
        inputs.append(context_input)

    context_layers = []
    for context_input in context_inputs:
        lstm = LSTM(64, return_sequences=True)(context_input)
        context_layers.append(lstm)

    context_concatenated = concatenate(context_layers)
    # context_reshaped = Reshape((context_concatenated.shape[1], 1))(context_concatenated)
    context_conv = Conv1D(filters=32, kernel_size=3, activation='relu', name='context_conv')(context_concatenated)
    # drop = Dropout(drop_amount)(context_conv)
    # batch_norm = BatchNormalization()(drop)
    # max_pool = MaxPooling1D()(batch_norm)
    # drop = Dropout(drop_amount)(max_pool)
    # batch_norm = BatchNormalization()(drop)
    context_layer = context_conv

    # Concatenate the outputs of the two branches
    combined = concatenate([pair_layer, context_layer], axis=1)
    flatten = Flatten()(combined)
    combined_dense = Dense(64)(flatten)
    # Output layer
    output = Dense(1, activation='sigmoid')(combined_dense)

    # Create the model with two inputs and one output
    model = Model(inputs=[inputs], outputs=output)

    # Compile the model
    opt = Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, decay=1e-5, amsgrad=False, clipvalue=0.5)
    model.compile(optimizer=opt, loss="binary_crossentropy", metrics=['mse'])

    return model

class to handle arguments instead of argument parser

In [10]:
class Args:
    def __init__(self, dataset, dataset_path, epochs, features, agents, frames, batch_size, reg, dropout, learning_rate, gmitre_calc):
        self.dataset = dataset
        self.dataset_path = dataset_path
        self.epochs = epochs
        self.features = features
        self.agents = agents
        self.frames = frames
        self.batch_size = batch_size
        self.reg = reg
        self.dropout = dropout
        self.learning_rate = learning_rate
        self.gmitre_calc = gmitre_calc


# Main

set argument constants

In [11]:
args = Args(
    dataset='eth',
    dataset_path='./datasets/ETH/seq_eth',
    epochs=40,
    features=4,
    agents=10,
    frames=10,
    batch_size=1024,
    reg=0.0000001,
    dropout=0.35,
    learning_rate=0.0001,
    gmitre_calc = True
)

load data

In [13]:
train, test, val, samples = load_dataset(
        '/content/drive/My Drive/datasets/{}_{}_{}'.format(args.dataset, args.frames, args.agents), args.agents,
        multi_frame=True)

initialise variables

In [14]:
tf.random.set_seed(0)
np.random.seed(0)
best_val_mses = []
best_val_f1s_one = []
best_val_f1s_two_thirds = []

create and train model

In [16]:
model = build_model(args.agents - 2, args.frames, args.features, 64, args.reg, args.dropout, args.learning_rate)

tensorboard = TensorBoard(log_dir='./logs')
early_stop = EarlyStopping(monitor='val_loss', patience=5)
history = ValLoss(val, args.dataset, args.dataset_path, samples, True, args.gmitre_calc)

model.fit(train[0], train[1], epochs=args.epochs, batch_size=args.batch_size,
              validation_data=(val[0], val[1]), callbacks=[tensorboard, early_stop, history])

Epoch 1/40
 5/42 [==>...........................] - ETA: 1s - loss: 0.6832 - mse: 0.2451

42/42 [==============================] - 41s 338ms/step - loss: 0.6583 - mse: 0.2329 - val_loss: 0.6466 - val_mse: 0.2275
Epoch 2/40
42/42 [==============================] - 6s 156ms/step - loss: 0.5996 - mse: 0.2050 - val_loss: 0.5627 - val_mse: 0.1876
Epoch 3/40
42/42 [==============================] - 8s 185ms/step - loss: 0.5211 - mse: 0.1698 - val_loss: 0.4564 - val_mse: 0.1422
Epoch 4/40
42/42 [==============================] - 7s 171ms/step - loss: 0.4301 - mse: 0.1321 - val_loss: 0.3578 - val_mse: 0.1057
Epoch 5/40
42/42 [==============================] - 7s 175ms/step - loss: 0.3498 - mse: 0.1024 - val_loss: 0.2986 - val_mse: 0.0887
Epoch 6/40
42/42 [==============================] - 6s 145ms/step - loss: 0.2904 - mse: 0.0824 - val_loss: 0.2578 - val_mse: 0.0757
Epoch 7/40
42/42 [==============================] - 5s 133ms/step - loss: 0.2499 - mse: 0.0697 - val_loss: 0.2257 - val_mse: 0.0659
Epoch 8/40
42/42 [==============================] - 12s 302ms/step - loss: 0.2152 - ms

keep track of evaluation metrics

In [17]:
save_model_data(args.dataset, args.reg, args.dropout, history, test, samples, True, gmitre_calc=args.gmitre_calc)

saving model to models/eth/pair_predictions_1
891/891 [==============================] - 8s 9ms/step
saved best val model as /best_val_model.h5
